In [ ]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Data cleaning

In [ ]:
train=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/4th PROJECT/Hazard_train.csv')

In [ ]:
test=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/4th PROJECT/Hazard_test_share.csv')

In [ ]:
cat_cols=train.select_dtypes(['object']).columns

In [ ]:
cat_cols

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train['data']='train'
test['data']='test'
all_data=pd.concat([train,test],0,sort=False)

In [ ]:
all_data.shape

In [ ]:
for col in cat_cols:
    k=all_data[col].value_counts()
    cats=k[k>=100].index[:-1]
    
    for cat in cats:
        name=col+'_'+cat
        all_data[name]=(all_data[col]==cat).astype(int)
        
    del all_data[col]

In [ ]:
all_data.shape

In [ ]:
x_train=all_data.drop(['Id','Hazard','data'],1)[all_data['data']=='train']
y_train=all_data['Hazard'][all_data['data']=='train']

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test=all_data.drop(['Id','Hazard','data'],1)[all_data['data']=='test']

# Data Meodelling 

In [ ]:
xgb_params = {  
                "learning_rate":[0.01,0.05,0.1,0.3,0.5],
                "gamma":[i/10.0 for i in range(0,5)],
                "max_depth": [2,3,4,5,6,7,8],
                "min_child_weight":[1,2,5,10],
                "max_delta_step":[0,1,2,5,10],
                "subsample":[i/10.0 for i in range(5,10)],
                "colsample_bytree":[i/10.0 for i in range(5,10)],
                "colsample_bylevel":[i/10.0 for i in range(5,10)],
                "reg_lambda":[1e-5, 1e-2, 0.1, 1, 100], 
                "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
                "scale_pos_weight":[1,2,3,4,5,6,7,8,9],
                "n_estimators":[100,500,700,1000]
             }

In [ ]:
xgb=XGBRegressor(objective='count:poisson')

In [ ]:
xgb_fit=GridSearchCV(xgb,n_jobs=-1,cv=10,scoring='neg_mean_absolute_error',
                                 param_grid=xgb_params)

In [ ]:
xgb_fit.fit(x_train,y_train)

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.5f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
report(xgb_fit.cv_results_,5)

In [ ]:
xgbre=XGBRegressor(**{'subsample': 0.7, 'scale_pos_weight': 3, 'reg_lambda': 0.1, 'reg_alpha': 1, 'n_estimators': 1000, 'min_child_weight': 10, 'max_depth': 5, 'max_delta_step': 5, 'learning_rate': 0.5, 'gamma': 0.1, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.7})

In [ ]:
xgbre.fit(x_train,y_train)

In [ ]:
submissions=pd.DataFrame({'Hazard':xgbre.predict(x_test)})

In [ ]:
submissions.to_csv('sample_submission.csv',index=False)